![banner-image](assets/banner-image.jpg)

<a id='top'></a>
# Project: Loan Default Prediction

## Table of Content
<ul>
    <li><a href='#intro'>Introduction</a></li>
    <li><a href='#gather'>Data Gathering</a></li>
    <li><a href='#explore'>Data Exploration</a></li>
    <li><a href='#merge'>Merging</a></li>
    <li><a href='#process'>Preprocessing</a></li>
    <li><a href='#engineer'>Feature Engineering</a></li>
    <li><a href='#imbalance'>Handling Imbalance</a></li>
    <li><a href='#prediction'>Predication</a></li>
    <li><a href='#results'>Results</a></li>
    <li><a href='#end'>End</a></li>
</ul>

<a id='intro'></a>

## Introduction

In [1]:
import pandas as pd
import numpy as np

<a id='gather'></a>

## Data Gathering

Collected data from Zindi Competiton - Nigeria dataset.

In [2]:
demo = pd.read_csv("data/train/traindemographics.csv")
perf = pd.read_csv("data/train/trainperf.csv")
prev = pd.read_csv("data/train/trainprevloans.csv")

<a id='explore'></a>

## Data Exploration

1. checking shape for demo, pref and prev datasets
2. checking for duplicate values in customer id in demo and pref. 

In [4]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  4346 non-null   object 
 1   birthdate                   4346 non-null   object 
 2   bank_account_type           4346 non-null   object 
 3   longitude_gps               4346 non-null   float64
 4   latitude_gps                4346 non-null   float64
 5   bank_name_clients           4346 non-null   object 
 6   bank_branch_clients         51 non-null     object 
 7   employment_status_clients   3698 non-null   object 
 8   level_of_education_clients  587 non-null    object 
dtypes: float64(2), object(7)
memory usage: 305.7+ KB


In [5]:
perf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4368 entries, 0 to 4367
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   customerid     4368 non-null   object 
 1   systemloanid   4368 non-null   int64  
 2   loannumber     4368 non-null   int64  
 3   approveddate   4368 non-null   object 
 4   creationdate   4368 non-null   object 
 5   loanamount     4368 non-null   float64
 6   totaldue       4368 non-null   float64
 7   termdays       4368 non-null   int64  
 8   referredby     587 non-null    object 
 9   good_bad_flag  4368 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 341.4+ KB


In [ ]:
demo.shape

(4346, 9)

In [4]:
perf.shape

(4368, 10)

In [5]:
prev.shape

(18183, 12)

In [6]:

demo['customerid'].nunique()

4334

In [7]:
perf['customerid'].nunique()

4368

In [8]:
prev['customerid'].nunique()

4359

<a id='merge'></a>

## Merge data

1. We merged all training data, demographics with performance

Performance (trainpref) is the MAIN dataset since it contains the TARGET (good_bad_flag)

In [6]:
df = perf.merge(demo, on="customerid", how="left")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  4376 non-null   object 
 1   systemloanid                4376 non-null   int64  
 2   loannumber                  4376 non-null   int64  
 3   approveddate                4376 non-null   object 
 4   creationdate                4376 non-null   object 
 5   loanamount                  4376 non-null   float64
 6   totaldue                    4376 non-null   float64
 7   termdays                    4376 non-null   int64  
 8   referredby                  589 non-null    object 
 9   good_bad_flag               4376 non-null   object 
 10  birthdate                   3277 non-null   object 
 11  bank_account_type           3277 non-null   object 
 12  longitude_gps               3277 non-null   float64
 13  latitude_gps                3277 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  4376 non-null   object 
 1   systemloanid                4376 non-null   int64  
 2   loannumber                  4376 non-null   int64  
 3   approveddate                4376 non-null   object 
 4   creationdate                4376 non-null   object 
 5   loanamount                  4376 non-null   float64
 6   totaldue                    4376 non-null   float64
 7   termdays                    4376 non-null   int64  
 8   referredby                  589 non-null    object 
 9   good_bad_flag               4376 non-null   object 
 10  birthdate                   3277 non-null   object 
 11  bank_account_type           3277 non-null   object 
 12  longitude_gps               3277 non-null   float64
 13  latitude_gps                3277 

In [11]:
prev['approveddate'] = pd.to_datetime(prev['approveddate'])
prev['creationdate'] = pd.to_datetime(prev['creationdate'])
prev['closeddate'] = pd.to_datetime(prev['closeddate'])
prev['firstduedate'] = pd.to_datetime(prev['firstduedate'])
prev['firstrepaiddate'] = pd.to_datetime(prev['firstrepaiddate'])

# repayment delay
prev['repayment_delay'] = (prev['firstrepaiddate'] - prev['firstduedate']).dt.days

# aggregation
agg_prev = prev.groupby("customerid").agg({
    "systemloanid": "count",            # number of previous loans
    "loanamount": ["mean", "max"],
    "totaldue": ["mean"],
    "termdays": ["mean", "max"],
    "repayment_delay": ["mean"],
})

agg_prev.columns = ["_".join(col) for col in agg_prev.columns]
agg_prev.head()

,systemloanid_count,loanamount_mean,loanamount_max,totaldue_mean,termdays_mean,termdays_max,repayment_delay_mean
customerid,,,,,,,
8a1088a0484472eb01484669e3ce4e0b,1,10000.000000,10000.0,11500.000000,15.000000,15,6.000000
8a1a1e7e4f707f8b014f797718316cad,4,17500.000000,30000.0,22375.000000,37.500000,60,-0.250000
8a1a32fc49b632520149c3b8fdf85139,7,12857.142857,20000.0,15214.285714,19.285714,30,-0.428571
8a1eb5ba49a682300149c3c068b806c7,8,16250.000000,30000.0,20300.000000,33.750000,60,-3.125000
8a1edbf14734127f0147356fdb1b1eb2,2,10000.000000,10000.0,12250.000000,22.500000,30,-4.000000


#### Merge aggregated previous-loan statistics

In [12]:
df = df.merge(agg_prev, on="customerid", how="left")

<a id='process'></a>

## Data Preprocessing

#### Handling Categorical

In [13]:
cat_cols = ["bank_account_type", "bank_name_clients", "employment_status_clients",
            "level_of_education_clients"]

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

#### Handle Missing Values

median for numeric

mode for categoricals

0 for aggregated features when there are no previous loans

In [14]:
X = df.drop("good_bad_flag", axis=1)
y = df["good_bad_flag"]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [15]:
y_train.value_counts(normalize=True)

good_bad_flag
Good    0.782286
Bad     0.217714
Name: proportion, dtype: float64

## Feature Engineering

In [16]:
X = df.drop("good_bad_flag", axis=1)
y = df["good_bad_flag"]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

## Check Imbalance

In [17]:
y_train.value_counts(normalize=True)

good_bad_flag
Good    0.782286
Bad     0.217714
Name: proportion, dtype: float64

<a id='results'></a>

## Results

<a id='end'></a> 

## End

<li><a href='#top'>Back to top</a></li>